In [8]:
pip install twilio

     ---------------------------------------- 0.0/77.6 kB ? eta -:--:--
     ---------------------------------------- 77.6/77.6 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   --- ------------------------------------ 0.2/1.8 MB 5.1 MB/s eta 0:00:01
   --------- ------------------------------ 0.4/1.8 MB 5.2 MB/s eta 0:00:01
   ------------------------ --------------- 1.1/1.8 MB 8.8 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 10.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/454.1 kB ? eta -:--:--
   -------------------------------- ------- 368.6/454.1 kB ? eta -:--:--
   ---------------------------------------- 454.1/454.1 kB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.1 kB ? eta -:--:--
   ---------------------------------------- 44.1/44.1 kB ? eta 0:00:00
   ---------------------------------------- 0.0/46.0 kB ? eta -:--:--
   ------------------------------------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# bot.py  –  WhatsApp AI shop  –  NO WooCommerce  –  Google-Sheet-as-JSON  –  India UPI
import os, json, csv, requests, redis, cv2, numpy as np, time, threading
from flask import Flask, request
import whisper, mediapipe as mp

app = Flask(__name__)
r = redis.from_url(os.getenv("REDIS_URL", "redis://localhost:6379"))

# --------------------  CONFIG  --------------------
ACCESS_TOKEN   = os.getenv("ACCESS_TOKEN")
PHONE_ID       = os.getenv("PHONE_ID")
VERIFY_TOKEN   = os.getenv("VERIFY_TOKEN")

RAZORPAY_KEY_ID = os.getenv("RAZORPAY_KEY_ID")
RAZORPAY_SECRET = os.getenv("RAZORPAY_SECRET")

SHOPIFY_DOMAIN = os.getenv("SHOPIFY_DOMAIN")
SHOPIFY_TOKEN  = os.getenv("SHOPIFY_STOREFRONT_TOKEN")
GOOGLE_KEY     = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID  = os.getenv("GOOGLE_CSE_ID")
OWNER_PHONE    = os.getenv("OWNER_PHONE")

# 1-MIN GOOGLE SHEET (zero keys) – best alternative to WooCommerce
SHEET_CSV = os.getenv("SHEET_CSV")  # publish-to-web link

# --------------------  WHATSAPP SEND  --------------------
from twilio.rest import Client

def send_whatsapp(to, kind="text", text=None, image=None, buttons=None):
    client = Client(os.getenv("TWILIO_ACCOUNT_SID"), os.getenv("TWILIO_AUTH_TOKEN"))
    from_ = os.getenv("TWILIO_WHATSAPP_NUMBER")
    to = f"whatsapp:{to}"

    if kind == "text":
        msg = client.messages.create(body=text, from_=from_, to=to)
    elif kind == "image":
        msg = client.messages.create(media_url=[image], from_=from_, to=to)
    elif kind == "buttons":
        # Twilio supports *List Picker* or *Quick Reply* – here we use *Quick Reply*
        choices = [{"type": "reply", "title": b["reply"]["title"], "description": "", "id": b["reply"]["id"]} for b in buttons]
        msg = client.messages.create(
            body=text,
            from_=from_,
            to=to,
            interactive={
                "type": "button",
                "action": {"buttons": [{"type": "reply", "title": b["reply"]["title"], "id": b["reply"]["id"]} for b in buttons]}
            }
        )
    return msg.sid

# --------------------  REAL-TIME DATA  --------------------
def fetch_shopify():
    if not SHOPIFY_TOKEN: return []
    q = """{ products(first: 20) {
        edges { node {
            id title vendor
            images(first:1) { edges { node { url } } }
            variants(first:1) { edges { node { price quantityAvailable } } }
        } }
    } }"""
    resp = requests.post(f"https://{SHOPIFY_DOMAIN}/api/2023-10/graphql.json",
                         json={"query": q},
                         headers={"X-Shopify-Storefront-Access-Token": SHOPIFY_TOKEN}).json()
    prods = []
    for e in resp["data"]["products"]["edges"]:
        n = e["node"]
        prods.append({
            "id": n["id"].split("/")[-1],
            "name": n["title"],
            "price": float(n["variants"]["edges"][0]["node"]["price"]),
            "stock": n["variants"]["edges"][0]["node"]["quantityAvailable"],
            "image": n["images"]["edges"][0]["node"]["url"] if n["images"]["edges"] else "",
            "vendor": "shopify"
        })
    return prods

def fetch_google_shopping(query):
    if not GOOGLE_KEY: return []
    url = f"https://www.googleapis.com/customsearch/v1?q={query}&key={GOOGLE_KEY}&cx={GOOGLE_CSE_ID}&num=5"
    resp = requests.get(url).json()
    prods = []
    for item in resp.get("items", []):
        if "₹" in item["title"]:
            try:
                price = int(item["title"].split("₹")[1].split()[0].replace(",", ""))
                prods.append({
                    "id": item["cacheId"][:10],
                    "name": item["title"][:70],
                    "price": price,
                    "stock": 999,
                    "image": item.get("pagemap", {}).get("cse_image", [{}])[0].get("src", ""),
                    "vendor": "google"
                })
            except: pass
    return prods

# *******  EASIEST WOO-COMMERCE REPLACEMENT  *******
def fetch_sheet():
    """Google Sheet → CSV → products (zero keys, 1-min setup)"""
    if not SHEET_CSV: return []
    r = requests.get(SHEET_CSV)
    r.encoding = 'utf-8'
    return list(csv.DictReader(r.text.splitlines()))

# --------------------  CACHE  --------------------
CACHE_FILE = "products.json"
def load_cache():
    return json.load(open(CACHE_FILE)) if os.path.exists(CACHE_FILE) else []
def save_cache(data):
    json.dump(data, open(CACHE_FILE, "w"), indent=2)

def refresh_products():
    shopify  = fetch_shopify()
    google   = fetch_google_shopping("sneakers")
    sheet    = fetch_sheet()          # ← NEW
    merged   = shopify + google + sheet
    save_cache(merged)
    return merged

PRODUCTS = refresh_products()
# background refresh every 5 min
def bg_refresh():
    while True:
        time.sleep(300)
        refresh_products()
threading.Thread(target=bg_refresh, daemon=True).start()

# --------------------  INDIA UPI PAYMENT  --------------------
RAZORPAY_KEY_ID = os.getenv("RAZORPAY_KEY_ID")
RAZORPAY_SECRET = os.getenv("RAZORPAY_SECRET")

def create_upi_order(phone, amount):
    url = "https://api.razorpay.com/v1/orders"
    auth = (RAZORPAY_KEY_ID, RAZORPAY_SECRET)
    payload = {
        "amount": int(amount * 100),
        "currency": "INR",
        "method": "upi",
        "receipt": f"{phone}_{int(time.time())}",
        "notes": {"phone": phone}
    }
    resp = requests.post(url, json=payload, auth=auth).json()
    order_id = resp["id"]
    # UPI intent deeplink (change to your VPA)
    upi_link = f"upi://pay?pa=YOUR_VPA@paytm&pn=YourShop&am={amount}&cu=INR&tn=Order{order_id[-6:]}"
    # QR code
    qr = requests.post("https://api.razorpay.com/v1/payments/qr_codes",
                       json={"type": "upi_qr", "name": order_id, "usage": "single_order"},
                       auth=auth).json()
    qr_b64 = qr["image_url"]
    return {"link": upi_link, "qr": qr_b64, "order_id": order_id}

def check_payment(order_id):
    auth = (RAZORPAY_KEY_ID, RAZORPAY_SECRET)
    resp = requests.get(f"https://api.razorpay.com/v1/orders/{order_id}/payments", auth=auth).json()
    for p in resp.get("items", []):
        if p["status"] == "captured":
            return True
    return False

# --------------------  CART  --------------------
def cart_key(phone): return f"cart:{phone}"
def add_cart(phone, product_id):
    r.hincrby(cart_key(phone), product_id, 1)
def show_cart(phone):
    items = r.hgetall(cart_key(phone))
    if not items:
        send_whatsapp(phone, "text", text="🛒 Cart is empty"); return 0
    total = 0; lines = []
    for pid, qty in items.items():
        p = next(p for p in PRODUCTS if p["id"] == pid.decode())
        sub = int(p["price"]) * int(qty); total += sub
        lines.append(f"{p['name']} x{qty.decode()} = ₹{sub}")
    lines.append(f"Total: ₹{total}")
    send_whatsapp(phone, "text", text="\n".join(lines))
    return total

# --------------------  AR TRY-ON  --------------------
mp_pose = mp.solutions.pose
def overlay_shoes(user_img_url, product_png_url):
    img   = cv2.imdecode(np.frombuffer(requests.get(user_img_url).content, np.uint8), cv2.IMREAD_COLOR)
    shoes = cv2.imdecode(np.frombuffer(requests.get(product_png_url).content, np.uint8), cv2.IMREAD_UNCHANGED)
    with mp_pose.Pose(static_image_mode=True) as pose:
        results = pose.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if not results.pose_landmarks: return None
        h, w = img.shape[:2]
        heel = int(results.pose_landmarks.landmark[30].y * h), int(results.pose_landmarks.landmark[30].x * w)
        shoes = cv2.resize(shoes, (160, 80))
        x, y = heel[1] - 80, heel[0] - 40
        alpha = shoes[:, :, 3] / 255.0
        for c in range(3):
            img[y:y + 80, x:x + 160, c] = alpha * shoes[:, :, c] + (1 - alpha) * img[y:y + 80, x:x + 160, c]
        ok, buf = cv2.imencode('.jpg', img)
        return buf.tobytes()

# --------------------  WEBHOOK  --------------------
@app.route("/webhook", methods=["GET"])
def verify():
    mode, token, challenge = request.args.get("hub.mode"), request.args.get("hub.verify_token"), request.args.get("hub.challenge")
    if mode == "subscribe" and token == VERIFY_TOKEN:
        return challenge
    return "OK"

@app.route("/webhook", methods=["POST"])
def hook():
    data = request.get_json()
    for entry in data["entry"]:
        for ch in entry["changes"]:
            if ch["field"] == "messages":
                msg = ch["value"]["messages"][0]
                phone = msg["from"]
                # 1. PHOTO -> TRY-ON
                if msg.get("type") == "image":
                    prod = [p for p in PRODUCTS if "shoes" in p["name"].lower()][0]
                    media_url = requests.get(f"https://graph.facebook.com/v18.0/{msg['image']['id']}", headers={"Authorization": f"Bearer {ACCESS_TOKEN}"}).json()["url"]
                    overlay = overlay_shoes(media_url, prod["image"])
                    if overlay:
                        imgr = requests.post("https://api.imgur.com/3/image", headers={"Authorization": "Client-ID 546c25a59c58ad7"}, files={"image": overlay}).json()
                        send_whatsapp(phone, "image", image=imgr["data"]["link"])
                    else:
                        send_whatsapp(phone, "text", text="Couldn’t find feet, retake photo.")
                # 2. VOICE
                elif msg.get("type") == "voice":
                    media_url = requests.get(f"https://graph.facebook.com/v18.0/{msg['voice']['id']}", headers={"Authorization": f"Bearer {ACCESS_TOKEN}"}).json()["url"]
                    voice = requests.get(media_url).content
                    with open("temp.ogg", "wb") as f: f.write(voice)
                    result = whisper.transcribe("temp.ogg")
                    text = result["text"].lower()
                    # save style pref
                    data = json.loads(r.get(f"style:{phone}") or '{"loves":[],"hates":[]}')
                    if "streetwear" in text or "baggy" in text: data["loves"].append("baggy")
                    if "skinny" in text: data["hates"].append("skinny")
                    r.set(f"style:{phone}", json.dumps(data))
                    send_whatsapp(phone, "text", text=f"👂 Heard: {text}")
                # 3. TEXT
                elif msg.get("type") == "text":
                    txt = msg["text"]["body"].lower()
                    if "shoes" in txt or "search" in txt:
                        p = PRODUCTS[0]
                        buttons = [{"type": "reply", "reply": {"id": f"add_{p['id']}", "title": "Add to cart"}}]
                        send_whatsapp(phone, "buttons", text=f"{p['name']} ₹{p['price']} (stock {p['stock']})", buttons=buttons)
                    elif "cart" in txt:
                        show_cart(phone)
                    elif "checkout" in txt:
                        total = show_cart(phone)
                        if total:
                            pay = create_upi_order(phone, total)
                            buttons = [{"type": "reply", "reply": {"id": f"pay_{pay['order_id']}", "title": "💳 Pay now"}}]
                            send_whatsapp(phone, "buttons", text=f"Total ₹{total}\nTap Pay to open any UPI app.", buttons=buttons)
                            send_whatsapp(phone, "image", image=pay["qr"])
                    else:
                        send_whatsapp(phone, "text", text="Menu: search | cart | checkout")
                # 4. BUTTON INTERACTIVE
                elif msg.get("type") == "interactive":
                    bid = msg["interactive"]["button_reply"]["id"]
                    if bid.startswith("add_"):
                        add_cart(phone, bid.split("_")[1])
                        send_whatsapp(phone, "text", text="✅ Added to cart")
                    elif bid.startswith("pay_"):
                        order_id = bid.split("_")[1]
                        paid = False
                        for _ in range(6):
                            if check_payment(order_id):
                                paid = True; break
                            time.sleep(5)
                        if paid:
                            send_whatsapp(phone, "text", text=f"✅ Payment successful! Order #{order_id[-6:]} confirmed.")
                            r.delete(cart_key(phone))
                        else:
                            send_whatsapp(phone, "text", text="⏰ Payment not completed. Tap Pay again or scan QR.")
    return "OK"
    from flask import send_from_directory

@app.route("/")  # home
def home():
    return send_from_directory("static", "index.html")

@app.route("/<path:filename>")  # privacy, terms, contact
def static_pages(filename):
    return send_from_directory("static", filename)

if __name__ == "_main_":
    app.run(host="0.0.0.0", port=int(os.environ.get("PORT", 8000)))